Author: Maxime Marin  
@: mff.marin@gmail.com

# Accessing IMOS data case studies: Walk-through and interactive session - Data Extraction

*'Ok Max, very cool but I really just want to save the data and do my own stuff'*

In this very short notebook, we show that once you have loaded your data, it is very easy to extract it out of jupyter onto a variety of format. 

## 1) Saving data

Let's bring back our dataset (new notebook), import some libraries and save it:


In [1]:
import sys
import os

sys.path.append('/home/jovyan/intake-aodn')
import intake_aodn
from intake_aodn.utils import save_netcdf
from  intake_aodn.easicache import set_easi
intake_aodn.cat = set_easi()

/home/jovyan/intake-aodn/intake_aodn/catalogs/aodn_refs.zip aodn_refs.zip s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/aodn_refs.zip
/home/jovyan/intake-aodn/intake_aodn/catalogs/main.yaml main.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/main.yaml
/home/jovyan/intake-aodn/intake_aodn/catalogs/aodn.yaml aodn.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/aodn.yaml
/home/jovyan/intake-aodn/intake_aodn/catalogs/nci.yaml nci.yaml s3://easihub-csiro-user-scratch/AROAWO7MSC2TWB77JYRIM:csiro-csiro-aad_mor582@csiro.au/nci.yaml


In [4]:
from intake_aodn.utils import get_local_cluster
client = get_local_cluster(n_workers=16,threads_per_worker=16)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/csiro-csiro-aad_mor582@csiro.au/proxy/8787/status,
Dashboard: /user/csiro-csiro-aad_mor582@csiro.au/proxy/8787/status,Workers: 16
Total threads: 256,Total memory: 24.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 16
Dashboard: /user/csiro-csiro-aad_mor582@csiro.au/proxy/8787/status,Total threads: 256
Started: Just now,Total memory: 24.00 GiB
Comm: tcp://127.0.0.1:45189,Total threads: 16
Dashboard: /user/csiro-csiro-aad_mor582@csiro.au/proxy/40911/status,Memory: 1.50 GiB
Nanny: tcp://127.0.0.1:34145,


2022-08-06 03:21:48,115 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,116 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,117 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,117 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,118 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,118 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,118 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,120 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,124 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,125 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,134 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-08-06 03:21:48,135 - distributed.nanny

In [5]:
import xarray as xr
data = xr.open_dataset('Example_Data.nc')

In [11]:
data.mean(dim=['longitude','latitude']).to_dataframe()

,sses_bias,l2p_flags,sst_dtime,sses_count,sea_surface_temperature,sses_standard_deviation,satellite_zenith_angle,quality_level,SST_at_depth
time,,,,,,,,,
1998-01-01 15:20:00,-0.101475,5253.504395,11932.692990,2.265913,296.159607,0.737372,60.279701,4.270384,25.338562
1998-01-02 15:20:00,-0.051116,5292.428223,15067.598784,2.630769,295.730682,0.701973,45.268234,4.468729,24.914230
1998-01-03 15:20:00,-0.175267,4419.891113,15698.489833,2.619184,293.553589,0.761112,31.984838,3.970143,23.203402
1998-01-04 15:20:00,-0.138857,4877.370117,15945.336588,3.805594,295.235107,0.569760,14.772025,4.760544,24.444250
1998-01-05 15:20:00,-0.182143,5034.127441,-475.088232,2.496050,296.151855,0.772807,24.727751,3.468808,25.105410
...,...,...,...,...,...,...,...,...,...
2020-12-27 15:20:00,0.144303,4818.007812,-2872.470453,3.820356,297.711243,1.339648,18.143103,4.866111,26.672453
2020-12-28 15:20:00,0.078716,3355.284180,-6595.130462,3.157634,296.945221,1.175429,8.761691,4.020315,25.825891
2020-12-29 15:20:00,0.096872,4589.134277,-2705.766044,2.755778,297.346313,1.154299,27.102179,4.036048,26.791414


In [6]:
%%time
df = data.mean(dim=['longitude','latitude']).to_dataframe()
df.to_csv('box_averaged.csv',float_format = '%.2f', na_rep = 'NaN')

CPU times: user 7.54 s, sys: 3.84 s, total: 11.4 s
Wall time: 14.1 s


We can of course save the data under a csv format. In the cell above, we first get the box-averaged timeseries and save it at a daily frequency.  
If we open the file in jupyter or excel, we will see that the file contains a number of row equals to the number of time entries in our dataset (~10k days). Now, if we wanted to save all pixels from our selected region, the number of rows would be multiplied by the number of pixels, which would quickly make the file very big and not practical to save in csv.

While saving large datasets into csv is possible, it would take a very long time, and excel for example would not display the complete dataset past 2^20 rows. Instead we can extract the data into a netcdf format, which is tailored for large 3D-4D datasets:

In [5]:
%%time
save_netcdf(data,filename = 'mySSTfile.nc')

CPU times: user 2.93 s, sys: 2.34 s, total: 5.27 s
Wall time: 16.8 s


It only took a little longer to save the daily data for all pixels into a netcdf format.

***

## 2) All in one

If users simply want to use these tools to access, download and extract data, this all could be done in a few lines of code:

In [10]:
client = get_local_cluster()
client

<Client: 'tcp://127.0.0.1:8786' processes=8 threads=8, memory=28.00 GiB>

In [ ]:
%%time
import sys
import os
sys.path.append('/home/jovyan/intake-aodn')
import intake_aodn
from intake_aodn.utils import get_local_cluster, save_netcdf

data=intake_aodn.cat.aodn_s3.SST_L3S_1d_ngt(startdt='2018-01-01',
                                          enddt='2020-12-31',
                                          cropto=dict(latitude=slice(-30,-28),longitude=slice(110,112))).read()
data
#save_netcdf(data,filename = 'mySSTfile.nc')